In [6]:
import pandas
from datetime import datetime, timedelta
import time

In [7]:
import pickle

In [8]:
from sklearn.externals import joblib

In [9]:
import numpy

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, roc_auc_score

In [11]:
chats = pandas.read_csv("../data/train/chats_train.csv").sort_values('timestamp')

In [12]:
news = pandas.read_csv("../data/train/news_train.csv")
onchain_EOS = pandas.read_csv("../data/train/onchain/EOS_train.csv")
onchain_TRX = pandas.read_csv("../data/train/onchain/TRX_train.csv")
prices = pandas.read_csv("../data/train/tickers_train.csv")

In [13]:
ticker = 'TRX'
delta = timedelta(hours=2)

In [14]:
#Найдем максимальную и минимальную даты в файле с ценами
min_date = datetime.today()
max_date = datetime.fromordinal(1)
# отфильтруем price по нашему токену и отсортируем по времени
tic_prices = prices.loc[prices['ticker']==ticker].sort_values('datetime',ascending=True)

In [15]:
for line in tic_prices['datetime']:
    curtime = datetime.strptime(line,"%Y-%m-%dT%H:%M:%SZ")
    if curtime > max_date:
        max_date = curtime
    if curtime < min_date:
        min_date = curtime

In [16]:
print(min_date, max_date)

(datetime.datetime(2018, 3, 1, 0, 0), datetime.datetime(2018, 4, 30, 22, 0))


In [17]:
Y = []
price_index = 0
cur_date = min_date
last_price = tic_prices['priceBtc'].iloc[0]
search_index = 0
search_step = 5
#search_width = 0.01
search_width = 0.02
while cur_date <= max_date:
    cur_price = tic_prices.iloc[price_index]['priceBtc']
    cur_y = 0
    for i in range(search_step):
        if price_index + i + 1 < tic_prices.shape[0]:
            new_price = tic_prices.iloc[price_index + i + 1]['priceBtc']
            if new_price > cur_price*(1 + search_width):
                cur_y = 1
                break
            elif new_price < cur_price*(1 - search_width):
                cur_y = 2
                break
    #print cur_date
    Y.append(cur_y)
#     price_time = datetime.strptime(cur_price['datetime'],"%Y-%m-%dT%H:%M:%SZ")
#     #print(price_index, price_time,cur_date, last_price)
#     if price_time == cur_date:
#         Y.append(1 if cur_price['priceBtc'] < last_price else 0)
#         last_price = cur_price['priceBtc']
#         price_index = price_index + 1
#     else:
#         Y.append(0) 
    price_index = price_index + 1
    cur_date = cur_date + delta


In [18]:
len(Y)

732

In [19]:
import pickle
output = open('../pickles/Y_trx.pickle', 'wb')
pickle.dump(Y, output)
output.close()

In [84]:
print cur_date, min_date

2018-05-01 00:00:00 2018-03-01 00:00:00


In [42]:
#import time
#import datetime
#d = datetime.date(2015,1,5)
#d= min_date + delta
#unixtime = time.mktime(d.timetuple())
#print unixtime

In [55]:
max_messages = 0
chats_index = 0
cur_chats_counter = 0
#Пропустим все тексты, которые появились раньше, чем за 2 часа до первой котировки
#first_text_timestamp = datetime.timestamp(min_date - delta) 
first_text_timestamp = time.mktime((min_date - delta).timetuple() )
print first_text_timestamp
#first_text_timestamp = datetime.timestamp(min_date - delta) 
while chats.iloc[chats_index]['timestamp'] < first_text_timestamp:
    chats_index = chats_index + 1
print(chats_index)
cur_day = min_date.day
cur_chats = []
cur_news = []
cur_date = min_date
#cur_date = min_date + delta
X_raw = []
X = []
while cur_date <= max_date:
    #while (chats.iloc[chats_index]['timestamp'] < datetime.timestamp(cur_date)):
    #while (chats.iloc[chats_index]['timestamp'] < time.mktime((cur_date).timetuple())):
    print datetime.fromtimestamp(chats.iloc[chats_index]['timestamp']), cur_date
    while (datetime.fromtimestamp(chats.iloc[chats_index]['timestamp']) < cur_date):
        print "cur_date: ", cur_date
        if chats.iloc[chats_index]['sender_is_bot']:            
            chats_index = chats_index + 1
            continue
        cur_chats.append(chats.iloc[chats_index][['chat_id', 'chat_title', 'emoji_list', 'sender_first_name', 'sender_id',
       'sender_is_bot', 'sender_last_name', 'sender_username', 'text']])
        chats_index = chats_index + 1
        cur_chats_counter = cur_chats_counter + 1
        
        #if cur_chats_counter > 200:
            #break
    if max_messages < cur_chats_counter:
        print(cur_chats_counter, cur_date)
        max_messages = cur_chats_counter
    X_raw.append(cur_chats.copy())
    cur_chats = []
    cur_chats_counter = 0
    cur_date = cur_date + delta
    if cur_date.day != cur_day:
        # print(cur_date)
        cur_day = cur_date.day

1519844400.0
0
2018-03-01 00:00:04 2018-03-01 00:00:00


AttributeError: 'list' object has no attribute 'copy'

In [13]:
X = []
for cur_X in X_raw:
    cur_str = ''
    for message in cur_X:
        cur_str = cur_str + str(message['text']) + ' '
    X.append(cur_str)

In [21]:
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=40, stratify=Y)
x_train = X[:658]
x_test = X[659:]



In [44]:
y_train = Y[:658]
y_test = Y[659:]
print(len(x_train), len(x_test), len(y_train), len(y_test))

658 73 658 73


In [15]:
vectorizer_char = TfidfVectorizer(max_features=40000,
                             min_df=5, 
                             max_df=0.5, 
                             analyzer='char', 
                             ngram_range=(1, 4))

vectorizer_char.fit(x_train);

tfidf_matrix_char_train = vectorizer_char.transform(x_train)
tfidf_matrix_char_test = vectorizer_char.transform(x_test)

lr_char = LogisticRegression(solver='sag', verbose=2, multi_class='multinomial')
#lr_char = RandomForestClassifier()



convergence after 24 epochs took 2 seconds
0.4246575342465753
0.9088145896656535


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s finished


In [53]:
tfidf_matrix_char_X = vectorizer_char.transform(X)

In [45]:
lr_char.fit(tfidf_matrix_char_train, y_train)

y_pred_char = lr_char.predict(tfidf_matrix_char_test)
print(accuracy_score(y_test, y_pred_char))
y_pred_train = lr_char.predict(tfidf_matrix_char_train)
print(accuracy_score(y_train, y_pred_train))

convergence after 22 epochs took 3 seconds
0.4657534246575342
0.8753799392097265


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s finished


In [50]:
joblib.dump(vectorizer_char, 'vectorizer_char.pkl') 

['vectorizer_char.pkl']

In [55]:
joblib.dump(tfidf_matrix_char_X, 'tfidf_matrix_char_X.pkl') 

['tfidf_matrix_char_X.pkl']

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=40, stratify=Y)
print(len(x_train), len(x_test), len(y_train), len(y_test))

658 74 658 74


In [ ]:
with open('X_raw.pickle', 'wb') as handle:
    pickle.dump(X_raw, handle, protocol=4)

In [99]:
with open('XY_train_XY_test.pickle', 'wb') as handle:
    pickle.dump([tfidf_matrix_char_train,tfidf_matrix_char_test, y_train, y_test ], handle, protocol=2)

In [48]:
acc = [0,0,0]
acc2 = [0,0]
for i in range (len(y_test)):
    if y_test[i] == y_pred_char[i]:
        acc[y_pred_char[i]] = acc[y_pred_char[i]] + 1
    if y_test[i] != 0:
        if y_pred_char[i] != 0:
            if y_test[i] == y_pred_char[i]:
                acc2[0] = acc2[0] + 1
            else:
                acc2[1] = acc2[1] + 1
            
print(acc, acc2, len(y_test))

[28, 5, 1] [6, 2] 73


In [41]:
ans = [0,0,0]
for i in range (len(y_test)):
    ans[y_test[i]] = ans[y_test[i]] + 1
print(ans, len(y_test))

[32, 23, 18] 73


In [32]:
y_pred_char

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 2, 2, 2, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 1, 0,
       2, 0, 0, 0, 0, 0, 0])

In [35]:
numpy.array(y_test) - y_pred_char

array([ 2,  2,  2,  1,  2,  2,  1,  1,  1,  1, -1,  0,  1,  0,  1,  0, -1,
       -2, -2, -2,  0, -1, -1,  0,  0,  0,  1,  1,  1,  1,  0,  0, -1,  2,
       -1,  1,  0,  0,  0,  0,  0, -2,  0, -2,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, -2,  0, -1,  0,  1, -1,  1,  1,  0,  2,  0,  2,
        2,  2,  0,  0,  0])

In [84]:
for i in range(50):
    print(tic_prices['priceBtc'].iloc[i], Y[i])

4.140250416666666e-06 0
4.152857083333333e-06 0
4.143457916666667e-06 1
4.179483333333334e-06 1
4.151290416666667e-06 1
4.143471666666667e-06 1
4.233758333333332e-06 0
4.274100000000001e-06 0
4.247275e-06 1
4.228935833333334e-06 1
4.1915191666666665e-06 1
4.263834166666667e-06 1
4.6015245833333335e-06 1
4.827294166666667e-06 2
4.593007916666667e-06 2
4.413669583333334e-06 1
4.654349166666667e-06 1
4.665066249999998e-06 1
4.740495833333333e-06 0
4.803226956521738e-06 2
4.769809583333333e-06 2
4.666515833333333e-06 1
4.711197916666668e-06 2
4.7777766666666675e-06 2
4.6991000000000005e-06 2
4.539133333333332e-06 2
4.486390416666667e-06 2
4.465064583333333e-06 2
4.47695875e-06 2
4.4055079166666665e-06 2
4.2579395833333325e-06 1
4.20390125e-06 1
4.267279166666667e-06 1
4.246307916666667e-06 1
4.36995875e-06 2
4.32069875e-06 0
4.3077099999999984e-06 0
4.2638720833333325e-06 0
4.262619583333334e-06 0
4.274446666666667e-06 0
4.2435183333333325e-06 1
4.27251375e-06 0
4.3142391666666676e-06 2
4.